In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
save_path = 'writers_with_img.csv'

In [ ]:
!pip install deep-translator

import requests
source = 'https://literature.britishcouncil.org'

from bs4 import BeautifulSoup
from deep_translator import GoogleTranslator
from bs4 import NavigableString, Tag
import pandas as pd
import re

In [ ]:
writers_df = pd.DataFrame(columns=['name', 'genres', 'birthplace', 'publishers', 'bio', 'critical_perspective', 'work', 'awards', 'image'])

for x in range(38):
  page = requests.get(source+'/writers/?start='+str(20*i))
  main_soup = BeautifulSoup(page.content, 'html.parser')
  writers = main_soup.find('div', class_='list-items -with-pagination').find_all('div', class_='list-item')

  for i in range (20):
    try:
      td = writers[i]

      url = td.find('a')['href']
      writer_page = requests.get(source+url)
      writer_soup = BeautifulSoup(writer_page.content, 'html.parser')

      name = writer_soup.find('h1', itemprop='name').text
      name_sin = GoogleTranslator(source='en', target='si').translate(name)
      # print (name, name_sin)

      if re.search('[a-zA-Z]', name_sin):
        print (name_sin)
        continue

      try:
        img = source+writer_soup.find('div', class_='col-xs-4 artist-img').find('img')['src']
      except:
        img = 'https://visualpharm.com/assets/128/User-595b40b85ba036ed117dc943.svg'
      
      genre_tags = writer_soup.find('ul', class_='artist-info__tags').find_all('li')
      genres = [i.text for i in genre_tags]
      genres_sin = ', '.join([GoogleTranslator(source='en', target='si').translate(i) for i in genres])

      birthplace = writer_soup.find('span', itemprop='birthPlace').text
      birthplace_sin = GoogleTranslator(source='en', target='si').translate(birthplace)
      
      publisher_tags = writer_soup.find_all('span', itemprop='worksFor')
      publishers_sin = ', '.join([GoogleTranslator(source='en', target='si').translate(i.text) for i in publisher_tags])

      bio_tags = writer_soup.find('div', itemprop='description').find_all('p')
      bio_sin = ' '.join([GoogleTranslator(source='en', target='si').translate(i.text) for i in bio_tags])

      crit_tags = writer_soup.find_all('div', class_='artist-page-waypoint')[0].find('div', class_='typ').find_all('p')
      crit_sin = ' '.join([GoogleTranslator(source='en', target='si').translate(i.text) for i in crit_tags])

      work_tags = writer_soup.find_all('div', class_='artist-page-waypoint')[1].find_all('div', class_='row')
      work_sin = ', '.join([GoogleTranslator(source='en', target='si').translate(i.find('div', itemprop='name').text)+' ('+i.find('div', itemprop='datePublished').text+')' for i in work_tags]) 

      award_tags = writer_soup.find_all('div', class_='artist-page-waypoint')[2].find_all('div', class_='row')
      awards_sin = ', '.join([GoogleTranslator(source='en', target='si').translate(i.find('div', class_='col-md-10').text)+' ('+i.find('div', class_='col-md-2').text+')' for i in award_tags]) 

      writers_df.loc[20*x+i] = [name_sin, genres_sin, birthplace_sin, publishers_sin, bio_sin, crit_sin, work_sin, awards_sin, img]
      print (x, i, 20*x+i, name, name_sin)

    except:
      print ('##Error', name)
      continue

In [ ]:
writers_df.to_csv(save_path)